In [1]:
push!(LOAD_PATH, "../src/")
import ConvexMomentumDynamics
using CoordinateTransformations
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh, Point
import ColorTypes: RGBA


using ConvexMomentumDynamics
using JuMP
using Gurobi
using DrakeVisualizer

In [15]:
println("starting CMD optimization test")
param = OptimizationParameters(num_timesteps = 30)
model = Model(solver=GurobiSolver(Presolve=0))
p = CentroidalDynamicsOptimizationProblem(model=model, param=param)
weights = OptimizationWeights(com_final_position = [0.0,0.,2.])
weights.com_final_position_weight = 100
weights.lin_momentum = 1.
weights.lin_momentum_final_weight = 10
weights.forces = 0
weights.convex_bounds = 1
initial_conditions = OptimizationInitialConditions()

com_nominal_position = initial_conditions.com_position

# create vector of contact points
contact_points = Vector{ContactPoint}()
r_foot_location = [-0.5,0.,0.]
l_foot_location = [0.5,0.,0.]

nominal_distance_to_com = norm(com_nominal_position - l_foot_location)

push!(contact_points, ContactPoint("r_foot",r_foot_location,
        nominal_distance_to_com))
push!(contact_points, ContactPoint("l_foot",l_foot_location,
        nominal_distance_to_com))

add_contact_points!(p, contact_points)
add_variables!(p)
add_dynamics_constraints!(p)
add_initial_condition_constraints!(p, initial_conditions)
add_costs!(p, weights)

# solve the model
status = solve(p.model);
soln = get_variable_solution_values(p.vars)
debug = compute_debug_info(p, soln);

starting CMD optimization test
Optimize a model with 459 rows, 1185 columns and 1719 nonzeros
Model has 549 quadratic objective terms
Model has 360 quadratic constraints
Presolve time: 0.05s
Presolved: 2260 rows, 2627 columns, 6161 nonzeros
Presolved model has 361 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Dense cols : 1
 Free vars  : 276
 AA' NZ     : 1.407e+04
 Factor NZ  : 4.706e+04 (roughly 2 MBytes of memory)
 Factor Ops : 1.256e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.04986942e+00  7.04986942e+00  8.12e+02 7.27e+02  8.72e+02     0s
   1  -1.02381831e+04 -5.89031481e+02  6.77e+02 5.31e+02  7.12e+02     0s
   2  -1.70213736e+04 -6.32221037e+02  6.49e+02 4.97e+02  6.72e+02     0s
   3  -3.76187128e+03 -1.08110911e+03  1.54e+02 3.80e+02  1.63e+02     0s
   4  -2.22589728e+02 -1.39950590e+03  1.03e+02 2.77e+

In [16]:
## Visualize CentroidalDynamicsState
println("visualizing centroidal dynamics state")
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();

options = CentroidalDynamicsVisualizerOptions(p.param)
cd_vis = CentroidalDynamicsVisualizer(p=p, soln=soln, debug=debug, options=options)
delete!(cd_vis.vis)

slider_playback(cd_vis)

Interact.Options{:SelectionSlider,Int64}(Signal{Int64}(15, nactions=1),"knot_point",15,"15",Interact.OptionDict(DataStructures.OrderedDict("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…),Dict(18=>"18",30=>"30",2=>"2",16=>"16",11=>"11",21=>"21",7=>"7",9=>"9",25=>"25",10=>"10"…)),Any[],Any[],true,"horizontal")

visualizing centroidal dynamics state


Set{Array{Symbol,1}}()

In [70]:
knot_point = 1
display(soln.forces[3,:,:])
# display(soln.lin_momentum[3,:])
# display(debug.l_cross_f[2,:,:])
# display(debug.l_cross_f[:,knot_point,1])
display(soln.l_cross_f_plus[:,knot_point,1])
display(soln.l_cross_f_plus[:,knot_point,2])


println("norm r_foot plus = ", norm(soln.l_cross_f_plus[:,knot_point,1]))
println("norm l_foot minus = ", norm(soln.l_cross_f_plus[:,knot_point,2]))
println()
println("norm r_foot plus = ", norm(soln.l_cross_f_minus[:,knot_point,1]))
println("norm l_foot minus = ", norm(soln.l_cross_f_minus[:,knot_point,2]))
# display(soln.forces[3,:,2])

# display(soln.lin_momentum[:,end])
# display(debug.orientation[:,end])

20×2 Array{Float64,2}:
 4.83545  4.86696
 4.86717  4.89848
 4.88252  4.91373
 4.88997  4.92115
 4.8936   4.92477
 4.89537  4.92653
 4.89623  4.92737
 4.89662  4.92777
 4.89679  4.92793
 4.89682  4.92795
 4.89672  4.92784
 4.89643  4.92755
 4.89579  4.92692
 4.89445  4.92559
 4.89169  4.92285
 4.88602  4.91722
 4.87536  4.90555
 4.8505   4.88262
 4.80092  4.8327 
 4.69578  4.72443

3-element Array{Float64,1}:
 6.09541 
 8.04931 
 0.140717

3-element Array{Float64,1}:
 6.17185  
 5.01758  
 0.0627021

norm r_foot plus = 10.097784163134177
norm l_foot minus = 7.95435507324212

norm r_foot plus = 7.532196525625464
norm l_foot minus = 9.675415217249792


In [4]:
# using Plots
# gr()
# pl = plot()
# plot!(pl, debug.ang_momentum[1,:], color=:blue, label="ang momentum")
# plot!(pl, debug.ang_momentum_dot[1,:], color=:red, label="ang momentum dot")
# plot!(title="angular momentum")

In [16]:
knot_point = 1
contact_idx = 1
l_cross_f_plus = soln.l_cross_f_plus[:,knot_point,contact_idx]
pos_convex = debug.pos_convex[:,knot_point,contact_idx]
l_x_f = debug.l_cross_f[:,knot_point,contact_idx]
display(l_cross_f_plus)
display(pos_convex)
display(l_cross_f_plus - pos_convex)
display(debug.ang_momentum_dot[:,knot_point])
display(soln.ang_momentum_dot[:,knot_point])

3-element Array{Float64,1}:
 6.45698 
 9.19838 
 0.249997

3-element Array{Float64,1}:
 6.45698 
 9.19838 
 0.249996

3-element Array{Float64,1}:
 -9.61212e-7
 -8.35616e-7
  6.21396e-7

3-element Array{Float64,1}:
 -1.5358e-5 
  2.45504   
  4.17135e-6

3-element Array{Float64,1}:
 -1.64439e-5
  2.45503   
 -2.99779e-6

In [80]:
r = [0.,0.,1.]
contact_point = [-0.5,0.,0.]
l = contact_point - r
l = [0.5,0,0.5]
l = l/norm(l)
println("l = ", l)

force_vertical = [0.,0.,1.]
force_com = l

pos_vert, neg_vert = difference_convex_functions_decomposition(l,force_vertical)
pos, neg = difference_convex_functions_decomposition(l,force_com)
display(pos_vert - neg_vert)
display(pos - neg)
println("vertical force ", norm(pos_vert) + norm(neg_vert))
println("force towards com ", norm(pos) + norm(neg))

3-element Array{Float64,1}:
  0.0     
 -0.707107
  0.0     

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

l = [0.707107,0.0,0.707107]
vertical force 1.3621806401054506
force towards com 1.224744871391589


In [49]:
display(pos - neg)
display(difference_convex_functions_decomposition(l,force_towards_com))
display(difference_convex_functions_decomposition(l,force_vertical))
display(cross(l,force_vertical))

3-element Array{Float64,1}:
 0.0        
 3.55271e-15
 0.0        

([24.26,30.325,6.065],[24.26,30.325,6.065])

([24.26,21.8725,0.0625],[24.26,26.7725,0.0625])

3-element Array{Float64,1}:
  0.0
 -4.9
 -0.0

In [5]:
l = [0,0,1]
typeof(1.0/norm(l))

Float64